# Imports

In [119]:
import pandas as pd
import numpy as np

#sklearn
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedShuffleSplit

# Utils
import random

SEED = 42

In [120]:
def set_seed():
    random.seed(SEED)
    np.random.seed(SEED)
set_seed()

In [2]:
!kaggle datasets download -d andrewmvd/fetal-health-classification  -f 'fetal_health.csv' -p ./datasets/

100%|████████████████████████████████████████| 223k/223k [00:00<00:00, 1.02MB/s]



# Cascading Classifier

In [153]:
class CascadingTreeClassifier:
    def __init__(self, cascading_depth, threshold):
        self.depth = cascading_depth
        self.threshold = threshold
        self.trees = self.generate_trees(cascading_depth)
        self.tree_counter = {i+1: 0 for i in range(cascading_depth)}
        self.descison_counter = {i+1: 0 for i in range(cascading_depth)}

    @staticmethod
    def generate_trees(cascading_depth):
        trees = [DecisionTreeClassifier(max_depth=i) for i in range(1, cascading_depth + 1)]
        return trees

    def fit(self, X, y):
        _ = [t.fit(X, y) for t in self.trees]
        return self


    def predict_instance(self, x):
        for i, tree in enumerate(self.trees):
            confidence_arr = tree.predict_proba(x.reshape(1, - 1))
            self.tree_counter[i + 1] += 1
            if confidence_arr.max() > self.threshold:
                self.descison_counter[i+1] += 1
                return confidence_arr
        
        self.descison_counter[i+1] += 1
        return confidence_arr

    def predict_proba(self, X):
        predictions = np.apply_along_axis(func1d=self.predict_instance, axis=1, arr=X)
        predictions = np.squeeze(predictions, axis=1)
        return predictions

    def predict(self, X):
        proba_pred = self.predict_proba(X)
        classes_pred = np.argmax(proba_pred, axis=1)
        return classes_pred

    


In [231]:
def generate_batch_indices(indices_range, batch_num, op):
    batches = []
    idxs = set()
    
    batch_size = np.ceil(indices_range / (batch_num - (batch_num - 1) * op)).astype(int)

    for _ in range(batch_num):
        b = []
        b_size_left = batch_size

        if len(idxs) < batch_size:
            b = list(idxs)
            b_size_left -= len(idxs)
            idxs = set()

        if not idxs:
            idxs = set(range(indices_range))


        b += random.sample(idxs, b_size_left)
        batches.append(b)
        
        idxs = idxs - set(b)
    
    return batches

In [216]:
class AdvancedCascadingTreeClassifier(CascadingTreeClassifier):
    def __init__(self, cascading_depth, threshold, overlapping_percentage):
        super().__init__(cascading_depth, threshold)
        self.overlapping_percentage = overlapping_percentage

    def fit(self, X, y):        
        batch_indices = generate_batch_indices(X.shape[0], self.depth, self.overlapping_percentage)
        
        for i, tree in enumerate(self.trees):
            tree.fit(X[batch_indices[i],:], y[batch_indices[i]])
            
        return self

In [217]:
class FeatureCascadingTreeClassifier(CascadingTreeClassifier):
    def __init__(self, cascading_depth, threshold, overlapping_percentage):
        super().__init__(cascading_depth, threshold)
        self.overlapping_percentage = overlapping_percentage
        

    def fit(self, X, y):
        self.feature_indices = generate_batch_indices(X.shape[1], self.depth, self.overlapping_percentage)

        for i, tree in enumerate(self.trees):
            tree.fit(X[:,self.feature_indices[i]], y)
            
        return self
    
    def predict_instance(self, x):
        for i, tree in enumerate(self.trees):
            confidence_arr = tree.predict_proba(x[self.feature_indices[i]].reshape(1, - 1))
            self.tree_counter[i + 1] += 1
            if confidence_arr.max() > self.threshold:
                self.descison_counter[i+1] += 1
                return confidence_arr
        
        self.descison_counter[i+1] += 1
        return confidence_arr

In [218]:
class CombineCascadingTreeClassifier(CascadingTreeClassifier):
    def __init__(self, cascading_depth, threshold, overlapping_percentage):
        super().__init__(cascading_depth, threshold)
        self.overlapping_percentage = overlapping_percentage
        

    def fit(self, X, y):
        self.feature_indices = generate_batch_indices(X.shape[1], self.depth, self.overlapping_percentage)
        batch_indices = generate_batch_indices(X.shape[0], self.depth, self.overlapping_percentage)

        for i, tree in enumerate(self.trees):
            tree.fit(X.take(batch_indices[i], axis=0).take(self.feature_indices[i], axis=1), y[batch_indices[i]])
            
        return self
    
    def predict_instance(self, x):
        for i, tree in enumerate(self.trees):
            confidence_arr = tree.predict_proba(x[self.feature_indices[i]].reshape(1, - 1))
            self.tree_counter[i + 1] += 1
            if confidence_arr.max() > self.threshold:
                self.descison_counter[i+1] += 1
                return confidence_arr
        
        self.descison_counter[i+1] += 1
        return confidence_arr

# Dataset 1

## Preprocessing

In [123]:
df_fetal = pd.read_csv(r'./datasets/fetal_health.csv')
df_fetal

,baseline value,accelerations,fetal_movement,uterine_contractions,light_decelerations,severe_decelerations,prolongued_decelerations,abnormal_short_term_variability,mean_value_of_short_term_variability,percentage_of_time_with_abnormal_long_term_variability,...,histogram_min,histogram_max,histogram_number_of_peaks,histogram_number_of_zeroes,histogram_mode,histogram_mean,histogram_median,histogram_variance,histogram_tendency,fetal_health
0,120.0,0.000,0.000,0.000,0.000,0.0,0.0,73.0,0.5,43.0,...,62.0,126.0,2.0,0.0,120.0,137.0,121.0,73.0,1.0,2.0
1,132.0,0.006,0.000,0.006,0.003,0.0,0.0,17.0,2.1,0.0,...,68.0,198.0,6.0,1.0,141.0,136.0,140.0,12.0,0.0,1.0
2,133.0,0.003,0.000,0.008,0.003,0.0,0.0,16.0,2.1,0.0,...,68.0,198.0,5.0,1.0,141.0,135.0,138.0,13.0,0.0,1.0
3,134.0,0.003,0.000,0.008,0.003,0.0,0.0,16.0,2.4,0.0,...,53.0,170.0,11.0,0.0,137.0,134.0,137.0,13.0,1.0,1.0
4,132.0,0.007,0.000,0.008,0.000,0.0,0.0,16.0,2.4,0.0,...,53.0,170.0,9.0,0.0,137.0,136.0,138.0,11.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2121,140.0,0.000,0.000,0.007,0.000,0.0,0.0,79.0,0.2,25.0,...,137.0,177.0,4.0,0.0,153.0,150.0,152.0,2.0,0.0,2.0
2122,140.0,0.001,0.000,0.007,0.000,0.0,0.0,78.0,0.4,22.0,...,103.0,169.0,6.0,0.0,152.0,148.0,151.0,3.0,1.0,2.0
2123,140.0,0.001,0.000,0.007,0.000,0.0,0.0,79.0,0.4,20.0,...,103.0,170.0,5.0,0.0,153.0,148.0,152.0,4.0,1.0,2.0
2124,140.0,0.001,0.000,0.006,0.000,0.0,0.0,78.0,0.4,27.0,...,103.0,169.0,6.0,0.0,152.0,147.0,151.0,4.0,1.0,2.0


In [155]:
X, y = df_fetal.iloc[:,:-1].values, df_fetal['fetal_health'].astype(int).values
y =  LabelEncoder().fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [210]:
set_seed()
cdt = CascadingTreeClassifier(15, 0.95)

In [211]:
cdt.fit(X_train, y_train)

In [212]:
train_probs = cdt.predict_proba(X_train)
train_preds = cdt.predict(X_train)
test_probs = cdt.predict_proba(X_test)
test_preds = cdt.predict(X_test)
print(f'acc test {accuracy_score(y_test, test_preds):.4f}')
print(f'log loss test: {log_loss(y_test, test_probs):.4f}')
print(f'acc train {accuracy_score(y_train, train_preds):.4f}')
print(f'log loss train: {log_loss(y_train, train_probs):.4f}')

acc test 0.9178
log loss test: 1.4900
acc train 0.9665
log loss train: 0.1477


In [222]:
set_seed()
cdt_advanced = AdvancedCascadingTreeClassifier(15, 0.95, 0.9)

In [223]:
cdt_advanced.fit(X_train, y_train)

In [180]:
train_probs = cdt_advanced.predict_proba(X_train)
train_preds = cdt_advanced.predict(X_train)
test_probs = cdt_advanced.predict_proba(X_test)
test_preds = cdt_advanced.predict(X_test)
print(f'acc test {accuracy_score(y_test, test_preds):.4f}')
print(f'log loss test: {log_loss(y_test, test_probs):.4f}')
print(f'acc train {accuracy_score(y_train, train_preds):.4f}')
print(f'log loss train: {log_loss(y_train, train_probs):.4f}')

acc test 0.9061
log loss test: 1.3895
acc train 0.9288
log loss train: 0.8918


In [225]:
set_seed()
cdt_features = FeatureCascadingTreeClassifier(15, 0.95, 0.4)

In [226]:
cdt_features.fit(X_train, y_train)

In [152]:
train_probs = cdt_features.predict_proba(X_train)
train_preds = cdt_features.predict(X_train)
test_probs = cdt_features.predict_proba(X_test)
test_preds = cdt_features.predict(X_test)
print(f'acc test {accuracy_score(y_test, test_preds):.4f}')
print(f'log loss test: {log_loss(y_test, test_probs):.4f}')
print(f'acc train {accuracy_score(y_train, train_preds):.4f}')
print(f'log loss train: {log_loss(y_train, train_probs):.4f}')

acc test 0.8779
log loss test: 3.6506
acc train 0.9800
log loss train: 0.0763


In [228]:
set_seed()
cdt_combine = CombineCascadingTreeClassifier(15, 0.95, 0.95)

In [229]:
cdt_combine.fit(X_train, y_train)

In [205]:
train_probs = cdt_combine.predict_proba(X_train)
train_preds = cdt_combine.predict(X_train)
test_probs = cdt_combine.predict_proba(X_test)
test_preds = cdt_combine.predict(X_test)
print(f'acc test {accuracy_score(y_test, test_preds):.4f}')
print(f'log loss test: {log_loss(y_test, test_probs):.4f}')
print(f'acc train {accuracy_score(y_train, train_preds):.4f}')
print(f'log loss train: {log_loss(y_train, train_probs):.4f}')

acc test 0.9014
log loss test: 1.5626
acc train 0.9382
log loss train: 0.7739
